In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
file = 'D:\\Dropbox\\My work\\krypl-project\\dataSplit\\01-priceMovement\\strategy_test.tsv'
data = read_tsv(file)
features = ['return', 'return_1', 'return_2', 'zscore_36_return',
       'zscore_288_return', 'zscore_36_volume', 'zscore_288_volume',
       'zscore_2016_volume', 'pma36', 'pma288', 'pma2016', 'vma36', 'vma288',
       'vma2016', 'ma_12_108', 'ma_36_288', 'ac36', 'ac288', 'vol36', 'vol288',
       'vol2016', 'dv_36_288', 'dv_288_2016']
data_manager = CurrencyDataManager(data['close'], data[features])
wallet = {'usdt': 1000}
contract_pair = ContractPair.new('usdt', 'btc')

# Load Model

In [3]:
data['date'].min()

'2017-10-05 00:00:00'

In [4]:
data['date'].max()

'2018-10-05 13:15:00'

In [5]:
from sklearn.externals import joblib
from modeling.gridsearch import GridSearch

path = "D:\\Dropbox\\My work\\krypl-project\\models\\01-priceMovement"
clf = load_model(f'{path}\\tree_selected_01.pkl')

In [8]:
from modeling.strategy import MLStrategy

DAY = (60 / 5) * 24
dm = deepcopy(data_manager)
exchange = BackTestExchange(dm, deepcopy(wallet), 0.0025)
strategy = MLStrategy(exchange, dm, contract_pair, 100, willing_loss=0.05, target_profit=0.01, clf=clf, max_hold_time=DAY)

# Report

In [9]:
from utils.evaluation import strategy_report

strategy_report(data[features], data['close'], strategy, contract_pair, wallet);

,usdt_hold,usdt_strategy
startAmount,"1,000.0000","1,000.0000"
numberOfTrades,1.0000,"1,258.0000"
totalProfit,55.8379,-549.0887
avgProfit,55.8379,-0.4365
winPercentage,100.0000,68.0445
avgWinTrade,55.8379,1.4022
avgLossTrade,0.0000,-4.3516
profitFactor,inf,0.6861
maxDrawdown,0.0000,55.9555
